In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os

# needed only for plotting
import plotly.express as px
import plotly.graph_objects as go

In [2]:
"""
This code is used to normalize the mapped and counted reads in all the ways presented in the paper (except for the DEseq2 normalization, which is done in R,see code in this folder). 

1. Normalize by ERCC spike-ins 
2. Calculate the RPK
3. Remove transcript below the detection limit
4. Normalize by downsampling to the lowest number of mapped reads
5. Normalize by downsampling to the lowest number of mapped reads and then calculating the RPK
"""

'\nThis code is used to normalize the mapped and counted reads in all the ways presented in the paper (except for the DEseq2 normalization, which is done in R,see code in this folder). \n\n1. Normalize by ERCC spike-ins \n2. Calculate the RPK\n3. Remove transcript below the detection limit\n4. Normalize by downsampling to the lowest number of mapped reads\n5. Normalize by downsampling to the lowest number of mapped reads and then calculating the RPK\n'

# Helper functions


In [3]:
# for htseq file reading:

def load_count_files(file_name_map,columns = ['ID','data']):
    # load all count files, and make a long-format pandas array with all data.
    # input is a list of dict with file location as keys and the sample names as values. All files are need to be in the htseq output format (tsv).
    # output is a pandas array of all data in one long table, where each row is a transcript from one sample.
    # 'file' column is added to indicate which file the data is from. 
    # columns is a list of column names for the output table (default: ['ID','data'])
    
    
    raw_reads = pd.DataFrame(columns=columns)
    for file in file_name_map.keys():
             
        this_pd = load_count_file(file,columns = columns)
        this_pd['file'] = file_name_map[file]
        raw_reads = pd.concat([raw_reads,this_pd])
    
    # remove statistics lines from htseq output
    stat_lines = ['__no_feature', '__ambiguous', '__too_low_aQual', '__not_aligned',
       '__alignment_not_unique']
    raw_reads = raw_reads[~raw_reads.isin(stat_lines).any(axis=1)]

    return raw_reads

def load_count_file(count_file, columns = ['ID','data']):
    # load the countsfile from location count_file 
    # input is a count file created by htseq (tsv) 
    # output is a pandas array of counts data with columns as specified in columns (default: ['ID','data'])
    raw_reads = pd.read_csv(count_file,sep='\t',header = None,names = columns)
    
    return raw_reads

# for ERCC normalization:

def get_ERCC_concentration(ERCC_info,mix):
    # mix = 1,2,none. get concentration data in attomoles/ul from the correct mix.
    col_name = 'concentration_in_Mix_' + str(int(mix)) + '_attomoles/ul'
    concentration_map = dict(zip(ERCC_info['ERCC_ID'],ERCC_info[col_name]))
    return concentration_map

def calculate_ERCC_attomoles_in_sample(RPK_ERCC,sample_data_loc,ERCC_info_loc):
    # This function calculates the ERCC attomoles in each sample according to the following formula: 
    # ERCC attomoles in sample = ERCC concentration * ul ERCC added to sample / ERCC dilution
    # The results are added to the RPK_ERCC dataFrame in the column 'ERCC_attomoles_in_sample'.

    # RPK_ERCC: a pandas array with the ERCC RPK data. This should be in long format, with the following columns:
        # file: sample name
        # ID: transcript name
        # RPK: RPK data
    # sample_data_loc should contain a csv file with the following columns:
        # file: sample name
        # mix: ERCC mix in sample (1 or 2)
        # ul_ERCC_added_to_sample: ul ERCC added to sample (from ERCC dilution from original stock)
        # ERCC_dilution: the ERCC dilution from the original stock.
            # for example if ERCC was diluted 1:40 and 1.3ul were added from this dilution to the sample 
            # we set: ul_ERCC_added_to_sample = 1.3, ERCC_dilution = 40.
        # may contain additional columns, this will not change the output.

    samples = list(set(RPK_ERCC['file']))
    # load samples metadata: how much ERCC was added to each sample, from which dilution, etc.
    sample_data_pd = pd.read_csv(sample_data_loc)

    # Load the ERCC metadata:
    ERCC_info = pd.read_csv(ERCC_info_loc)

    # define the ERCC mix in each sample:
    mix_map = dict(zip(sample_data_pd['file'],sample_data_pd['mix']))

    # define the amount of ERCC added to each sample (ul):
    ul_ERCC_added_to_sample_map = dict(zip(sample_data_pd['file'],sample_data_pd['ul_ERCC_added_to_sample']))

    # define the ERCC dilution from the original stock: (1:40 would be 40)
    ERCC_dilution_map = dict(zip(sample_data_pd['file'],sample_data_pd['ERCC_dilution']))

    # load ERCC concentration data for each sample (from the right mix):
    concentration_map = {sample: get_ERCC_concentration(ERCC_info,mix) for sample,mix in mix_map.items()}

    def add_concentration_data_to_pd(RPK_ERCC,concentration_map):
        # add ERCC concentration data to the RPK_ERCC dataFrame.
        # concentration_map is a dict with sample names as keys and ERCC concentration data as values.
        RPK_ERCC['file_ID'] = RPK_ERCC['file'] + '_' + RPK_ERCC['ID']
        new_concentration_map = {}
        for sample,concentration in concentration_map.items():
            new_concentration_map.update({sample + '_' + ERCC: concentration[ERCC] for ERCC in concentration.keys()})
        RPK_ERCC['ERCC_concentration'] = RPK_ERCC['file_ID'].map(new_concentration_map)
        RPK_ERCC = RPK_ERCC.drop(columns=['file_ID'])
        return RPK_ERCC

    # add all the metadata to the ERCC dataFrame:
    RPK_ERCC['ul_ERCC_added_to_sample'] = RPK_ERCC['file'].map(ul_ERCC_added_to_sample_map)
    RPK_ERCC['ERCC_dilution'] = RPK_ERCC['file'].map(ERCC_dilution_map)
    RPK_ERCC = add_concentration_data_to_pd(RPK_ERCC,concentration_map)

    # calculate the ERCC attomoles in each sample:
    RPK_ERCC['ERCC_attomoles_in_sample'] = RPK_ERCC['ERCC_concentration'] * RPK_ERCC['ul_ERCC_added_to_sample'] / RPK_ERCC['ERCC_dilution']
    return RPK_ERCC

def find_ERCC_normalization_parameters(ERCC_RPK,x_DL_map,ignore_ERCCs=[]):
    # find the normalization parameters for each sample (file) using the ERCC spike-ins.
    # This function linear fits: log(RPK) = m*log(ERCC_attomoles_in_sample) + c
    # Where RPK is measured in the sample, and ERCC_attomoles_in_sample is the ERCC attomoles in the sample as calculated by the mix known properties (see the function calculate_ERCC_attomoles_in_sample).
    # ERCC_RPK: a pandas array with the ERCC RPK data. This should be in long format, with the following columns:
        # file: sample name
        # ID: transcript name
        # data: RPK data
        # ERCC_attomoles_in_sample: ERCC attomoles in sample (calculated by the function calculate_ERCC_attomoles_in_sample)
    # x_DL_map: a dict with sample names as keys and the chosen lower limit of the ERCC_attomoles_in_sample data obove wich to do the fit as values.
    
    # ignore_ERCCs: list of transcript names to ignore, for example: ['ERCC-00002','ERCC-00003']. 

    # output: parameter_map: a dict with sample names as keys and the fit parameters as values. 
    # if the fit fails, the parameters are set to 0 and the r2 to the error message.

    def log_transform_and_fit(x_data,y_data):
        
        # log transforms x_data and y_data, and does a linear fit 
        # input: x_data,y_data numpy arrays. 
        # output: m,c are the fit parameters: log(y_data) = m*log(x_data)+c
        #        r2 is r-squared of the fit

        x_fit = x_data
        y_fit = y_data
        x_fit = x_fit[y_fit > 0] # remove transcript if no reads were detected
        y_fit = y_fit[y_fit > 0]


        XYlog = np.vstack([np.log(x_fit.astype('double')),np.ones(len(x_fit))]).T
        y_data_log = np.log(y_fit.astype('double'))
        model, resid = np.linalg.lstsq(XYlog, y_data_log, rcond=None)[:2]
        m, c = model
        r2 = 1 - resid / (y_data_log.size * y_data_log.var())
        r2 = r2[0]
        
        return m, c, r2


    # remove ERCCs to ignore:
    ERCC_pd_for_fit = ERCC_RPK[~ERCC_RPK['ID'].isin(ignore_ERCCs)].copy()
    
    # find normalization parameters for each sample (file)
    parameter_map = {} 
    samples = list(set(ERCC_pd_for_fit['file']))       
    for sample in samples:
        this_data = ERCC_pd_for_fit[ERCC_pd_for_fit['file']==sample]
        x_data = this_data['ERCC_attomoles_in_sample']
        y_data = this_data['RPK']
        
        # only use data above the chosen lower limit:
        y_data = y_data[this_data['RPK']>x_DL_map[sample]]
        x_data = x_data[this_data['RPK']>x_DL_map[sample]]
        
        try:
            m, c, r2 = log_transform_and_fit(x_data,y_data)
            parameter_map[sample] = {'m':m, 'c':c, 'r2': r2}        
        except Exception as e:
            # if the fit fails, set the parameters to 0 and the r2 to the error message.
            parameter_map[sample] = {'m':0, 'c':0, 'r2': str(e)}

    return parameter_map 

def plot_ERCC_linear_fit(RPK_ERCC,file,parameter_map,x_DL_map,ignore_ERCCs=[]):
    
    # plot the ERCC RPK Vs ERCC attomoles in sample, and add the fit.
    # RPK_ERCC: a pandas array with the ERCC RPK data. This should be in long format, with the following columns:
        # file: sample name
        # ID: transcript name
        # data: RPK data
        # ERCC_attomoles_in_sample: ERCC attomoles in sample (calculated by the function calculate_ERCC_attomoles_in_sample)
    # file: the sample name to plot
    # parameter_map: a dict with sample names as keys and the fit parameters as values. This is the output of the function find_ERCC_normalization_parameters.
    # x_DL_map: a dict with sample names as keys and the chosen lower limit of the ERCC_attomoles_in_sample data obove wich the fit was done as values.
    # ignore_ERCCs: list of transcript that were ignored, for example: ['ERCC-00002','ERCC-00003'].

    # output: a plotly figure with the ERCC RPK Vs ERCC attomoles in sample, and the fit, in log-log scale. the fit parameters are added to the figure. the ignored ERCCs are marked in red.

    def plot_ERCC_RPK_Vs_ERCC_attomoles_in_sample(RPK_ERCC):
        fig = px.scatter(RPK_ERCC,x='ERCC_attomoles_in_sample',
                        y='RPK',log_x=True,log_y=True,
                        color = 'in_fit',
                     hover_name='ID',width=2000,height=600)
        #plot(fig)
        fig.update_layout(font_size=24)
        return fig
    
    def my_fig_annotation_box(fig,text,alignto='left',
                          x=0,xanchor='left'):
        # text is a list of strings with text lines
        i = 0
        for line in text:
            fig.add_annotation(
                    text=line,
                    align=alignto,
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    xanchor = xanchor,
                    x=x,
                    y=1-0.1*i,
                    bordercolor='black',
                    borderwidth=1)
            i+=1
        return fig 

    ERCC_pd = RPK_ERCC[RPK_ERCC['file'] == file]
    ERCC_pd['in_fit'] = ERCC_pd['ID'].apply(lambda x: 'in fit' if x not in ignore_ERCCs else 'not in fit')
    
    
    fig = plot_ERCC_RPK_Vs_ERCC_attomoles_in_sample(ERCC_pd)
    
    max_x = max(ERCC_pd['ERCC_attomoles_in_sample'])
    
    # add fit line:
    m = parameter_map[file]['m']
    c = parameter_map[file]['c']
    r2 = parameter_map[file]['r2']
    x=np.array([x_DL_map[file],max_x])
    fig.add_trace(go.Scatter(x=x,y=np.exp(c)*x**m,mode='lines',name=file))
    
    if not isinstance(r2, str):
        r2 = format(r2, '.2f')
    
    # add fit info
    fit_text = ['fit of log(y) Vs log(x), paremeters: log(y) = m*log(x)+c',
                'fit in the range ERCC attomoles in sample [' + format(x_DL_map[file],'.2f') + ',' + format(max_x, '.2f') + ']',
                'the slop in log scale: m = ' + format(m, '.2f'),
                'the intercept in log scale: c = ' + format(c, '.2f'),'',
                'log(y) = m*log(x)+c --> y = e^c * x^m',
                'the slope in linear scale: e^c = ' + format(np.exp(c), '.2E'),
                'R^2 = ' + r2,'',
                'ERCCs with no reads detected', ' not fitted and not shown']
    fig = my_fig_annotation_box(fig,fit_text,x=0.4,xanchor='left')
    fig.update_xaxes(range=[-3, 15])
    
    return fig

def get_length_from_gff(gff_file_loc):
    # Load the gff file:
    gff_file = gffpd.read_gff3(gff_file_loc)
    gff_file.df = gff_file.df[gff_file.df['type']=='gene']
    gff_file.df['name'] = gff_file.df['attributes'].apply(lambda x: x.split('Name=')[1].split(';')[0])

    # get the transcript lengths (if a gene has multiple annotated transcripts, the sum of the lengths of all transcripts is taken. This happens in our gff only of RelA, since it has an insertion):
    length_map = {gene:0 for gene in gff_file.df['name']}
    for row in gff_file.df.iterrows():
        this_row = row[1]
        this_gene = this_row['name']
        this_length = this_row['end'] - this_row['start']+1
        length_map[this_gene] += this_length
    transcript_lengths = pd.DataFrame.from_dict(length_map,orient='index')
    transcript_lengths['ID'] = transcript_lengths.index
    transcript_lengths.columns = ['length','ID']
    return transcript_lengths

def get_product_info_from_gff(gff_file_loc):
    # Load the gff file:
    gff_file = gffpd.read_gff3(gff_file_loc)
    gff_file_CDS = gff_file.df[gff_file.df['type']=='CDS']
    gff_file_CDS['product'] = gff_file_CDS['attributes'].apply(lambda x: x.split('product=')[1].split(';')[0])
    gff_file_CDS['Parent'] = gff_file_CDS['attributes'].apply(lambda x: x.split('Parent=')[1].split(';')[0] )
    gff_file_CDS['Source: Product'] = gff_file_CDS['source'] + ': ' + gff_file_CDS['product']
    ID_product_map = dict(zip(gff_file_CDS['Parent'],gff_file_CDS['Source: Product']))
    gff_file_gene = gff_file.df[gff_file.df['type']=='gene']
    gff_file_gene = gff_file_gene[gff_file_gene['attributes'].str.contains('ID=')]
    gff_file_gene['ID'] = gff_file_gene['attributes'].apply(lambda x: x.split('ID=')[1].split(';')[0])
    gff_file_gene['Name'] = gff_file_gene['attributes'].apply(lambda x: x.split('Name=')[1].split(';')[0])
    ID_name_map = dict(zip(gff_file_gene['ID'],gff_file_gene['Name']))
    gene_product_map = {}
    for id in ID_product_map.keys():
        #print(id)
        
        if id in ID_name_map.keys():
            gene_product_map[ID_name_map[id]] = ID_product_map[id]
        
    
    return gene_product_map

# Help functions for the resampling normalization:
def create_resample_data(data,read_name_col,data_col):
    # data is a dataframe with a column of read names and a column of data
    # read_name_col is the name of the column with the read names
    # data_col is the name of the column with the data
    data_for_sampling = []
    read_names = data[read_name_col]
    for ele in read_names:
        data_ele = list(data[data[read_name_col]==ele][data_col])[0]
        # make a list of ele repeated as the number of times the read repeats
        l2append = [ele]*data_ele
        data_for_sampling = data_for_sampling + l2append
    return data_for_sampling

# a function to subsample the data of one sample to the min_reads
def resample(data,read_name_col,data_col,samples):
    # data is a dataframe with a column of read names and a column of data
    # read_name_col is the name of the column with the read names
    # data_col is the name of the column with the data
    # samples is the number of samples to take

    # create a list of all the reads in the data, each read repeated as the number of times it was counted
    data_for_sampling = create_resample_data(data,read_name_col,data_col)

    # resample the data
    resampled_data = pd.DataFrame(columns=[read_name_col,data_col])
    resampled_data[read_name_col] = data[read_name_col].copy()
    #resampled_data[data_col] = 0
    transcripts = list(set(resampled_data[read_name_col]))
    count_dict = {t:0 for t in transcripts}
    for s in range(samples):
        chosen = random.choice(data_for_sampling)
        count_dict[chosen] = count_dict[chosen]+1
        #resampled_data.loc[resampled_data[read_name_col]==chosen,data_col] = resampled_data[resampled_data[read_name_col]==chosen][data_col] 
    resampled_data[data_col] = resampled_data[read_name_col].map(count_dict)
    return resampled_data

# Load the raw counts

In [4]:
# set the experiment name:
Experiment_name = 'Experiment1'
#Experiment_name = 'Experiment2'
# Experiment_name = 'subsample'
#Experiment_name = 'Exponential'
#Experiment_name = 'Time_in_SHX'
# This code will go to the folder: ..\map_count_outputs\"Experiment_name"\counts\ and load all count files.

# get the names of all count files:
counts_file_path = '..\\map_count_outputs\\'+Experiment_name+'\\counts\\'
count_files = glob(counts_file_path+'*.tsv')
file_sample_map = {file:os.path.basename(file).split('.')[0] for file in count_files if ('reverse' not in file) and ('intergenic' not in file) and ('pseudogene' not in file) and ('notstranded' not in file)} # No need for reverse mapping


# load the count files:
raw_reads = load_count_files(file_sample_map,['ID','data'])
# separate the ERCC spike-ins from the rest of the data:
raw_reads_ERCC = raw_reads[raw_reads['ID'].str.contains('ERCC')]
raw_reads = raw_reads[~raw_reads['ID'].str.contains('ERCC')]



raw_reads['file'].unique()

array(['CASP_biorep1a', 'CASP_biorep1b', 'CASP_biorep1c', 'CASP_biorep2',
       'CASP_biorep3', 'CASP_biorep4', 'Disrupted_biorep1a',
       'Disrupted_biorep1b', 'Disrupted_biorep1c', 'Disrupted_biorep2',
       'Disrupted_biorep3', 'Disrupted_biorep4'], dtype=object)

In [5]:

# add gene product information from the gff file (can be commented out if not needed):
import gffpandas.gffpandas as gffpd
import pandas as pd
gff_file_loc = '..\Genome_files\gffFile_combined.gff'
product_map = get_product_info_from_gff(gff_file_loc)
raw_reads['source: product'] = raw_reads['ID'].map(product_map)




C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gff_file_CDS['product'] = gff_file_CDS['attributes'].apply(lambda x: x.split('product=')[1].split(';')[0])
C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gff_file_CDS['Parent'] = gff_file_CDS['attributes'].apply(lambda x: x.split('Parent=')[1].split(';')[0] )
C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:260: 

# Calculate the ERCC normalization

To clculate the ERCC normalization:
1. Calculate the RPK (reads per kilobase) for each gene and each ERCC spike-in according to the formula:
$$RPK=\frac{reads\ aligned\ to\ transcript}{transcript\ length / 1000}$$
2. Find the attomols of ERCC added to each sample according to the formula:
$$attomoles\ per\ sample\ ERCC=\frac{\mu l\ ERCC\ added \cdot attomole\ per\ \mu l\ in\ ERCC\ mix}{dilution\ of\ ERCC\ Mix}$$
(For example, in our setup we added 1.3 ul of ERCC mix1 to each sample from a 1:40 dilution of mix1. In that case $\mu l\ ERCC\ added=1.3$ and $dilution\ of\ ERCC\ Mix=40$. The attomole per $\mu l$ in ERCC mix1for each transcript is taken from the ERCC datasheet)

3. Linear Fit log(RPK) Vs log(ERCC attomoles in sample), to get the parameters of normalization for each sample, with the parameters:
$$\log (RPK\ ERCC) =m\cdot\log (attomoles\ per\ sample\ ERCC) +c$$
The fit is for data in range attomole per sample >0.01. Two ERCCs that were always problematic (never on the linear trend) were removed from the fit.
    We also Plot RPK as a function of attomoles in sample for each sample for ERCC transcripts. ERCCs removed from the fit are marked in red.

4. Use m,c for each sample to find the attomoles per sample of all transcripts:
$$attomoles\ per\ sample\ =e^{-\frac{c}{m}}\cdot (RPK)^{\frac{1}{m}}$$

5. To estimate the number of transcripts per cell, we normalize each sample by the number of cells in that sample (calculated using CFU counts). To find an estimate the number of cells in each sample, we plated the cells before sample collection, and found the CFU per ml. By knowing the volume of the sample, we then calculate the number of cells in the sample. The number of cells in each sample is given in the CSV file in the folder "Experiment metadata" under the column "Number of cells in sample":

$$attomoles\ per\ cell\ =\frac{attomoles\ per\ sample\ }{cells\ in\ sample}$$

In [6]:
# get transcript lengths from the gff file (uncomment if needed, the length are already saved in the file transcript_lengths.csv):
"""
import gffpandas.gffpandas as gffpd
import pandas as pd

gff_file_loc = '..\Genome_files\gffFile_combined.gff'
transcript_lengths = get_length_from_gff(gff_file_loc)
transcript_lengths.to_csv('normalizion_help_functions_and_info_tables/transcript_lengths.csv')
"""


# Get the transcript lengths:
transcript_lengths = pd.read_csv('normalizion_help_functions_and_info_tables/transcript_lengths.csv')
transcript_lengths_map = dict(zip(transcript_lengths['ID'],transcript_lengths['length']))

# Calculate the RPK:
RPK = raw_reads.copy()
RPK['length'] = RPK['ID'].map(transcript_lengths_map)
RPK['RPK'] = RPK['data']/(RPK['length']/1000)
RPK = RPK.drop(columns=['data'])

# Calculate the RPK for the ERCC spike-ins:
RPK_ERCC = raw_reads_ERCC.copy()
RPK_ERCC['length'] = RPK_ERCC['ID'].map(transcript_lengths_map)
RPK_ERCC['RPK'] = RPK_ERCC['data']/(RPK_ERCC['length']/1000)
RPK_ERCC = RPK_ERCC.drop(columns=['data'])




In [7]:
# calculate the ERCC attomoles in each sample:
sample_data_loc = '..//Experiment metadata//'+Experiment_name+'.csv'
ERCC_info_loc = 'normalizion_help_functions_and_info_tables/ERCC_info.csv'

RPK_ERCC = calculate_ERCC_attomoles_in_sample(RPK_ERCC,sample_data_loc,ERCC_info_loc)

# Calculate the ERCC normalization parameters for each sample:

# set the limit of ERCC attomoles in sample above which the fit will be done: 
samples = list(set(RPK_ERCC['file']))
x_DL_map = {sample:0.01 for sample in samples}

# set the ERCCs to ignore in the fit:
ignore_ERCCs = ['ERCC-00054']

parameter_map = find_ERCC_normalization_parameters(RPK_ERCC,x_DL_map,ignore_ERCCs)
parameter_map

{'CASP_biorep4': {'m': 0.9994356852708378,
  'c': 6.739996111223491,
  'r2': 0.9680413717596051},
 'CASP_biorep2': {'m': 0.9846284629786842,
  'c': 6.83732021982415,
  'r2': 0.9719791901673394},
 'Disrupted_biorep2': {'m': 0.9770832006868032,
  'c': 5.844344425251942,
  'r2': 0.9702719711572231},
 'Disrupted_biorep4': {'m': 0.9494078148771999,
  'c': 5.207922711298116,
  'r2': 0.9703451462658139},
 'Disrupted_biorep1a': {'m': 0.9912814866885004,
  'c': 6.133681687752506,
  'r2': 0.9671374429389816},
 'CASP_biorep1a': {'m': 0.9926203592150387,
  'c': 7.186296097096157,
  'r2': 0.9728729693023916},
 'CASP_biorep3': {'m': 0.9893746210619769,
  'c': 6.817603351757707,
  'r2': 0.9678673015541115},
 'Disrupted_biorep1c': {'m': 0.9645119868848481,
  'c': 6.092382275228463,
  'r2': 0.963027935622581},
 'CASP_biorep1c': {'m': 0.9810653979662296,
  'c': 7.2286311842887905,
  'r2': 0.9731861715697676},
 'Disrupted_biorep3': {'m': 0.9560903734214022,
  'c': 5.786210027857781,
  'r2': 0.95328059711

In [8]:
parameter_map_pd = pd.DataFrame()
parameter_map_pd['file'] = list(parameter_map.keys())
parameter_map_pd['m'] = [parameter_map[file]['m'] for file in parameter_map_pd['file']]
parameter_map_pd['c'] = [parameter_map[file]['c'] for file in parameter_map_pd['file']]
parameter_map_pd.to_csv('exp.csv')
parameter_map_pd

,file,m,c
0,CASP_biorep4,0.999436,6.739996
1,CASP_biorep2,0.984628,6.837320
2,Disrupted_biorep2,0.977083,5.844344
3,Disrupted_biorep4,0.949408,5.207923
4,Disrupted_biorep1a,0.991281,6.133682
5,CASP_biorep1a,0.992620,7.186296
6,CASP_biorep3,0.989375,6.817603
7,Disrupted_biorep1c,0.964512,6.092382
8,CASP_biorep1c,0.981065,7.228631
9,Disrupted_biorep3,0.956090,5.786210


In [9]:
# plot the fit for each sample:
# make a folder for the plots:
import os
if not os.path.exists('ERCC_normalizion_plots/'+Experiment_name):
    os.makedirs('ERCC_normalizion_plots/'+Experiment_name)
for file in parameter_map.keys():
    fig = plot_ERCC_linear_fit(RPK_ERCC,file,parameter_map,x_DL_map,ignore_ERCCs)
    # If you don't want to show the plots, comment out the next line:
    fig.show()
    # Save the plots (comment out if you don't want to save the plots. The html version is interactive):
    fig.write_html('ERCC_normalizion_plots/'+Experiment_name+'/'+file+'.html')
    #fig.write_image('ERCC_normalizion_plots/downsampled/'+file+'.png')


C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ERCC_pd['in_fit'] = ERCC_pd['ID'].apply(lambda x: 'in fit' if x not in ignore_ERCCs else 'not in fit')


C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\adiro\AppData\Local\Temp\ipykernel_32200\2061613557.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
# Set the lower limit of dtection to RPK = 2
# This limit is ~ 2 read per transcript. This is where the ERCC attomoles in sample is ~ 0.01 attomoles. The ERCCs show a linear relation to the RPK above this limit.
# Higher limits can be used, and this dose not change the main results regarding the biological noise.

"""
# get a list of transcripts with RPK < LLD in each sample:
lower_than_LLD = RPK[RPK['RPK']<2]
lower_than_LLD = lower_than_LLD.groupby(['file','ID']).count().reset_index()[['file','ID']]
lower_than_LLD['file_ID'] = lower_than_LLD['file'] + '_' + lower_than_LLD['ID']
#lower_than_LLD.to_csv('lower_then_LLD.csv')

# remove transcripts with RPK < LLD in each sample:
# dop rows whith ID and file in "lower_than_LLD"
RPK['file_ID'] = RPK['file'] + '_' + RPK['ID']
RPK = RPK[~RPK['file_ID'].isin(lower_than_LLD['file_ID'])]
RPK = RPK.drop(columns=['file_ID'])
"""


'\n# get a list of transcripts with RPK < LLD in each sample:\nlower_than_LLD = RPK[RPK[\'RPK\']<2]\nlower_than_LLD = lower_than_LLD.groupby([\'file\',\'ID\']).count().reset_index()[[\'file\',\'ID\']]\nlower_than_LLD[\'file_ID\'] = lower_than_LLD[\'file\'] + \'_\' + lower_than_LLD[\'ID\']\n#lower_than_LLD.to_csv(\'lower_then_LLD.csv\')\n\n# remove transcripts with RPK < LLD in each sample:\n# dop rows whith ID and file in "lower_than_LLD"\nRPK[\'file_ID\'] = RPK[\'file\'] + \'_\' + RPK[\'ID\']\nRPK = RPK[~RPK[\'file_ID\'].isin(lower_than_LLD[\'file_ID\'])]\nRPK = RPK.drop(columns=[\'file_ID\'])\n'

In [11]:
# save the raw data and RPK data (after removing transcripts with RPK < LLD):

# create save directory:
if not os.path.exists('Normalized_data_'+Experiment_name):
    os.makedirs('Normalized_data_'+Experiment_name)

"""
# remove transcripts with RPK < LLD in each sample:
raw_reads['file_ID'] = raw_reads['file'] + '_' + raw_reads['ID']
raw_reads = raw_reads[~raw_reads['file_ID'].isin(lower_than_LLD['file_ID'])]
raw_reads = raw_reads.drop(columns=['file_ID'])
"""

# Convert to wide format and save to file, for easier reading:
raw_reads_wide = raw_reads.pivot(index='ID', columns='file', values='data')
raw_reads_wide.to_csv('Normalized_data_'+Experiment_name+'/raw_counts_'+Experiment_name+'.csv')
raw_reads_ERCC_wide = raw_reads_ERCC.pivot(index='ID', columns='file', values='data')
raw_reads_ERCC_wide.to_csv('Normalized_data_'+Experiment_name+'/raw_counts_ERCC_'+Experiment_name+'.csv')


# add the biotype information to the RPK data:
k12_biotypes = pd.read_csv('normalizion_help_functions_and_info_tables/k12_biotype_map.csv')
k12_biotypes_map = dict(zip(k12_biotypes['gene'],k12_biotypes['biotype']))
k12_biotypes_map['KanR'] = 'protein_coding'
k12_biotypes_map['TetR'] = 'protein_coding'
k12_biotypes_map['mCherry'] = 'protein_coding'
RPK['biotype'] = RPK['ID'].map(k12_biotypes_map)

RPK_wide = RPK.pivot(index='ID', columns='file', values='RPK')
RPK_wide.to_csv('Normalized_data_'+Experiment_name+'/RPK_'+Experiment_name+'.csv')


In [12]:
# Normalize the data:

# add the normalization parameters to the RPK data:
for param in ['m','c']:
    RPK[param] = RPK['file'].map({file:parameter_map[file][param] for file in parameter_map.keys()})
# calculate the normalized data:
RPK['ERCC normalized data attomoles per sample'] = np.exp(-RPK['c']/RPK['m']) * RPK['RPK']**(1/RPK['m'])
# Change the units from attomoles to molecules in sample:
#RPK['ERCC normalized data moles per sample'] = np.array(RPK['ERCC normalized data attomoles per sample'])*10**-18 # there is a bug in panadas for large numbers, must go through numpy
RPK['ERCC normalized data molecules per sample'] = np.array(RPK['ERCC normalized data attomoles per sample'])*10**-18*6.022*10**23

# save the ERCC normalized data to file (In molecules per sample):
# MPS_wide = RPK.pivot(index='ID', columns='file', values='ERCC normalized data molecules per sample')
# MPS_wide.to_csv('MPS_'+Experiment_name+'.csv')

In [13]:
# Estimated molecules per cell:

cells_per_sample = pd.read_csv(sample_data_loc)[['file','Number of cells in sample']]
cells_per_sample = dict(zip(cells_per_sample['file'],cells_per_sample['Number of cells in sample']))
RPK['cells_per_sample'] = RPK['file'].map(cells_per_sample)
RPK['ERCC normalized data molecules per cell'] = RPK['ERCC normalized data molecules per sample']/RPK['cells_per_sample']
MPC_wide = RPK.pivot(index='ID', columns='file', values='ERCC normalized data molecules per cell')

# add gene product information from the gff file (can be commented out if not needed):
MPC_wide['source: product'] = MPC_wide.index.map(product_map) # add gene product information (can be commented out if not needed)

# save the ERCC normalized data (molecules per cell) to file:
MPC_wide.to_csv('Normalized_data_'+Experiment_name+'\ERCC_normalized_data_molecules_per_cell_'+Experiment_name+'.csv')

In [14]:
MPC_wide = pd.read_csv('Normalized_data_'+Experiment_name+'\ERCC_normalized_data_molecules_per_cell_'+Experiment_name+'.csv')
MPC_wide

,ID,CASP_biorep1a,CASP_biorep1b,CASP_biorep1c,CASP_biorep2,CASP_biorep3,CASP_biorep4,Disrupted_biorep1a,Disrupted_biorep1b,Disrupted_biorep1c,Disrupted_biorep2,Disrupted_biorep3,Disrupted_biorep4,source: product
0,CDN75_RS05665,0.004137,0.020120,0.004436,0.021790,0.033236,0.017340,0.473856,0.299095,0.414786,1.344872,0.459744,0.100328,Protein Homology: PTS galactitol transporter s...
1,CDN75_RS10700,0.000042,0.000167,0.000053,0.000294,0.000248,0.000159,0.117020,0.069055,0.090894,0.229346,0.099941,0.010302,Protein Homology: fimbria/pilus outer membrane...
2,CDN75_RS13515,0.000036,0.000180,0.000073,0.000147,0.000301,0.000205,0.251960,0.134363,0.206564,0.446093,0.244650,0.017306,Protein Homology: IS3-like element IS150 famil...
3,CDN75_RS17410,0.000422,0.001518,0.000383,0.001829,0.002283,0.001526,0.395608,0.238574,0.352153,1.151709,0.377003,0.062795,Protein Homology: tyrosine-type recombinase/in...
4,HQ24_RS00100,0.000078,0.000102,0.000075,0.000292,0.000267,0.000161,0.169024,0.095294,0.148993,0.384503,0.167676,0.009906,Protein Homology: IS1-like element IS1A family...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,zraR,0.000216,0.000788,0.000153,0.000569,0.001078,0.000692,0.300584,0.180751,0.260738,0.407184,0.263340,0.024570,Protein Homology: sigma-54-dependent response ...
4318,zraS,0.000070,0.000244,0.000037,0.000169,0.000247,0.000221,0.197519,0.110685,0.167672,0.227798,0.173997,0.009025,Protein Homology: two-component system sensor ...
4319,zupT,0.004298,0.017547,0.003752,0.017655,0.028416,0.018569,0.618500,0.379020,0.575162,2.558121,0.607345,0.141373,Protein Homology: zinc transporter ZupT
4320,zur,0.000441,0.001236,0.000419,0.001370,0.002292,0.001433,0.085805,0.056804,0.091826,0.435873,0.096870,0.016019,Protein Homology: zinc uptake transcriptional ...


In [15]:
MPC_wide.set_index('ID',inplace=True)
sample = 'CASP_biorep1a'
a = MPC_wide[[sample]].copy()
a = a[a[sample]>0]
# remove nan values:
a = a.dropna()
a

# remove inf values:
a = a[a[sample]!=np.inf]
# remove -inf values:
a = a[a[sample]!=-np.inf]
a[sample+' log10'] = a[sample].apply(lambda x: np.log10(x))
a

k12_biotypes = pd.read_csv('normalizion_help_functions_and_info_tables/k12_biotype_map.csv')
k12_biotypes_map = dict(zip(k12_biotypes['gene'],k12_biotypes['biotype']))
k12_biotypes_map['KanR'] = 'protein_coding'
k12_biotypes_map['TetR'] = 'protein_coding'
k12_biotypes_map['mCherry'] = 'protein_coding'
a['biotype'] = a.index.map(k12_biotypes_map)
a

a = a[~a['biotype'].isna()]
px.histogram(a,x=sample+' log10',color='biotype',nbins=50,barmode='overlay')


In [17]:
# since there are a lot rRNA, tRNA and ncRNA in the data, we will remove them from the data when calculating the mean molecules per cell:

total_MPC = RPK[RPK['biotype']=='protein_coding'].groupby('file')['ERCC normalized data molecules per cell'].sum().reset_index()
total_MPC['replicate'] = total_MPC['file'].apply(lambda x: x.split('_')[-1])
total_MPC['state'] = total_MPC['file'].apply(lambda x: x.split('_')[0])
total_MPC_stats = total_MPC.groupby(['state'])['ERCC normalized data molecules per cell'].mean().reset_index()
total_MPC_stats = total_MPC_stats.rename(columns={'ERCC normalized data molecules per cell':'mean molecules per cell'})
total_MPC_stats['std molecules per cell'] = total_MPC.groupby(['state'])['ERCC normalized data molecules per cell'].std().reset_index()['ERCC normalized data molecules per cell']
# bar plot of the mean molecules per cell:
fig = px.bar(total_MPC_stats,x='state',y='mean molecules per cell',error_y='std molecules per cell',color='state',barmode='group',log_y=True,width=600)
fig.show()

In [75]:
total_MPC_stats

,state,mean molecules per cell,std molecules per cell
0,CASP,16.856400,20.345888
1,Disrupted,570.121301,248.273372
2,EXP,2987.021438,2052.571476


In [55]:
RPK.groupby('file')['ERCC normalized data molecules per cell'].sum()

# Calculate the downsampling normalization

This normalizes the data by subsampaling the raw reads.
1. Find the sample with the lowest number of mapped reads
2. For each sample, subsample the reads to the number of reads in the sample with the lowest number of reads

In [18]:
# Calculate the total number of raw reads in each sample:
min_totl_reads = raw_reads.groupby('file')['data'].sum().min()
print('The smalest number of reads in a sample is: ' + str(min_totl_reads/10**6) + 'M reads')



The smalest number of reads in a sample is: 0.355808M reads


In [19]:
# resample the data to the smallest sample size:

samples = raw_reads['file'].unique()
import time
import random


# resample the data to the min_totl_reads for each sample
resampled_data = pd.DataFrame(columns=raw_reads.columns)
for sample in samples:
    
    #count the minutes it takes to resample
    start = time.time()
    
    this_sample = raw_reads[raw_reads['file']==sample].copy()
    
    this_sample_resampled = resample(this_sample,'ID','data',min_totl_reads)
    stop=time.time()
    # convert to minutes
    print('Time it took to resample: ' + sample+ ' is: '+ str((stop-start)/60) + ' minutes')


    this_sample_resampled['file'] = sample
    resampled_data = pd.concat([resampled_data,this_sample_resampled])

    # This run could be long, so uncomment the next line if you want to save the data to file after each sample:
    # resampled_data.to_csv('Normalized_data_'+Experiment_name+'/downsampled_data_raw_reads_'+Experiment_name+'.csv')

resampled_data_wide = resampled_data.pivot(index='ID', columns='file', values='data')
resampled_data_wide.to_csv('Normalized_data_'+Experiment_name+'/downsampled_data_raw_reads_'+Experiment_name+'.csv')

Time it took to resample: CASP_biorep1a is: 0.4377278049786886 minutes
Time it took to resample: CASP_biorep1b is: 1.010517172018687 minutes
Time it took to resample: CASP_biorep1c is: 0.49286088546117146 minutes
Time it took to resample: CASP_biorep2 is: 0.81421826283137 minutes
Time it took to resample: CASP_biorep3 is: 1.0314512411753336 minutes
Time it took to resample: CASP_biorep4 is: 0.9517118453979492 minutes
Time it took to resample: Disrupted_biorep1a is: 1.1312514623006185 minutes
Time it took to resample: Disrupted_biorep1b is: 0.9783519705136617 minutes
Time it took to resample: Disrupted_biorep1c is: 1.1729070663452148 minutes
Time it took to resample: Disrupted_biorep2 is: 1.228201448917389 minutes
Time it took to resample: Disrupted_biorep3 is: 1.0167391220728557 minutes
Time it took to resample: Disrupted_biorep4 is: 0.08475191990534464 minutes


In [20]:
resampled_data.groupby('file')['data'].sum()

file
CASP_biorep1a         355808
CASP_biorep1b         355808
CASP_biorep1c         355808
CASP_biorep2          355808
CASP_biorep3          355808
CASP_biorep4          355808
Disrupted_biorep1a    355808
Disrupted_biorep1b    355808
Disrupted_biorep1c    355808
Disrupted_biorep2     355808
Disrupted_biorep3     355808
Disrupted_biorep4     355808
Name: data, dtype: object

In [21]:
# resample the ERCC data to the smallest sample size:

samples = raw_reads_ERCC['file'].unique()
import time
import random

min_totl_reads = raw_reads_ERCC.groupby('file')['data'].sum().min()

# resample the data to the min_totl_reads for each sample
resampled_data_ERCC = pd.DataFrame(columns=raw_reads_ERCC.columns)
for sample in samples:
    
    #count the minutes it takes to resample
    start = time.time()
    
    this_sample = raw_reads_ERCC[raw_reads_ERCC['file']==sample].copy()
    
    this_sample_resampled = resample(this_sample,'ID','data',min_totl_reads)
    stop=time.time()
    # convert to minutes
    print('Time it took to resample: ' + sample+ ' is: '+ str((stop-start)/60) + ' minutes')


    this_sample_resampled['file'] = sample
    resampled_data_ERCC = pd.concat([resampled_data_ERCC,this_sample_resampled])

    # This run could be long, so uncomment the next line if you want to save the data to file after each sample:
    # resampled_data.to_csv('Normalized_data_'+Experiment_name+'/downsampled_data_raw_reads_'+Experiment_name+'.csv')

resampled_data_ERCC_wide = resampled_data_ERCC.pivot(index='ID', columns='file', values='data')
resampled_data_ERCC_wide.to_csv('Normalized_data_'+Experiment_name+'/downsampled_ERCC_data_raw_reads_'+Experiment_name+'.csv')

Time it took to resample: CASP_biorep1a is: 0.020451549688975015 minutes
Time it took to resample: CASP_biorep1b is: 0.011573199431101482 minutes
Time it took to resample: CASP_biorep1c is: 0.020639340082804363 minutes
Time it took to resample: CASP_biorep2 is: 0.01584238608678182 minutes
Time it took to resample: CASP_biorep3 is: 0.015537516276041666 minutes
Time it took to resample: CASP_biorep4 is: 0.014946345488230388 minutes
Time it took to resample: Disrupted_biorep1a is: 0.01056591272354126 minutes
Time it took to resample: Disrupted_biorep1b is: 0.01582869291305542 minutes
Time it took to resample: Disrupted_biorep1c is: 0.010354034105936686 minutes
Time it took to resample: Disrupted_biorep2 is: 0.009707117080688476 minutes
Time it took to resample: Disrupted_biorep3 is: 0.00834198792775472 minutes
Time it took to resample: Disrupted_biorep4 is: 0.007475856939951579 minutes


# Calculate the RPK after downsampling normalization

In [22]:
# get transcript lengths from the gff file (uncomment if needed, the length are already saved in the file transcript_lengths.csv):
"""
import gffpandas.gffpandas as gffpd
import pandas as pd

gff_file_loc = '..\Genome_files\gffFile_combined.gff'
transcript_lengths = get_length_from_gff(gff_file_loc)
transcript_lengths.to_csv('normalizion_help_functions_and_info_tables/transcript_lengths.csv')
"""

# Get the transcript lengths:
transcript_lengths = pd.read_csv('normalizion_help_functions_and_info_tables/transcript_lengths.csv')
transcript_lengths_map = dict(zip(transcript_lengths['ID'],transcript_lengths['length']))

# Calculate the RPK after resampling:
resample_RPK = resampled_data.copy()
resample_RPK['length'] = resample_RPK['ID'].map(transcript_lengths_map)
resample_RPK['RPK'] = resample_RPK['data']/(resample_RPK['length']/1000)

# save the RPK after downsampling data to file:
resample_RPK_wide = resample_RPK.pivot(index='ID', columns='file', values='RPK')
resample_RPK_wide.to_csv('Normalized_data_'+Experiment_name+'/RPK_after_downsample_'+Experiment_name+'.csv')


In [23]:
# Calculate the RPK after resampling for the ERCC:
resample_RPK_ERCC = resampled_data_ERCC.copy()
resample_RPK_ERCC['length'] = resample_RPK_ERCC['ID'].map(transcript_lengths_map)
resample_RPK_ERCC['RPK'] = resample_RPK_ERCC['data']/(resample_RPK_ERCC['length']/1000)

# save the RPK after downsampling data to file for the ERCC:
resample_RPK_ERCC_wide = resample_RPK_ERCC.pivot(index='ID', columns='file', values='RPK')
resample_RPK_ERCC_wide.to_csv('Normalized_data_'+Experiment_name+'/RPK_after_downsample_ERCC'+Experiment_name+'.csv')


# Get the mapping statistics

In [ ]:
from glob import glob


In [20]:
def get_mapped_percent(metadata):
    mapped_percent = metadata.iloc[6].values[0]
    return float(mapped_percent.split('%')[0].split('(')[1])

def get_total_reads(metadata):
    total_reads = metadata.iloc[0].values[0]
    return int(total_reads.split(' ')[0])


# get all the mapped percents and total reads
mapped_metadata = pd.DataFrame(columns=['file','sample','mapped_percent'])
name_map = dict(zip(raw_reads['file'],raw_reads['file']))
for sample in name_map.keys():
    #print(sample)
    if sample in ['transcript','gene_biotype','on_plasmid']:
        continue

    aligned_folder = counts_file_path.split('\\')[:-2] + ['aligned'] + ['log']
    # concatenate the list of strings to a single string with a separator '\\':
    aligned_folder = '\\'.join(aligned_folder)
    log_file_locs = aligned_folder + '\\' + sample + '_samtools_flagstat.txt'
    metadata = pd.read_csv(log_file_locs,header=None)
    mapped_metadata = pd.concat([mapped_metadata,pd.DataFrame({'file':sample,'sample':sample,'mapped_percent':get_mapped_percent(metadata),'total_reads':get_total_reads(metadata),'total_reads[M]':get_total_reads(metadata)/10**6},index=[0])])

# add the total reads to the mapped to ERCC data and percent of ERCC data
ERCC_sum_raw_reads = raw_reads_ERCC.groupby('file')['data'].sum().reset_index()
ERCC_sum_raw_reads.rename(columns={'data':'ERCC_raw_reads','file':'sample'}, inplace=True)
mapped_metadata = pd.merge(mapped_metadata, ERCC_sum_raw_reads, on='sample')

"""
mapped_metadata['ERCC_mapped_percent'] = mapped_metadata['ERCC_raw_reads']/mapped_metadata['total_reads']*100
# total reads without ERCC
#mapped_metadata['total_reads_minus_total_reads_ERCC_mapped'] = mapped_metadata['total_reads']*mapped_metadata['mapped_percent'] - mapped_metadata['ERCC_raw_reads']
"""
mapped_metadata

,file,sample,mapped_percent,total_reads,total_reads[M],ERCC_raw_reads
0,EXP_biorep1a,EXP_biorep1a,93.51,15156272.0,15.156272,5758917
1,EXP_biorep1b,EXP_biorep1b,95.34,16363445.0,16.363445,5673464
2,EXP_biorep1c,EXP_biorep1c,95.29,13657354.0,13.657354,8038146
3,EXP_biorep2,EXP_biorep2,93.12,17136552.0,17.136552,5642581
4,EXP_biorep3,EXP_biorep3,96.37,14950780.0,14.950780,6656597
